<a href="https://colab.research.google.com/github/a-winders/Capstone_ATW/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

Mounts Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Importing Datasets

Imports different scraped .csv movie files

In [ ]:
movie_agg = pd.read_csv('/content/gdrive/MyDrive/Bellarmine/CS430_Data_Files/archive/movies_metadata.csv')
score = pd.read_csv('/content/gdrive/MyDrive/Bellarmine/CS430_Data_Files/archive/ratings.csv')
cast = pd.read_csv('/content/gdrive/MyDrive/Bellarmine/CS430_Data_Files/archive/credits.csv')
tags = pd.read_csv('/content/gdrive/MyDrive/Bellarmine/CS430_Data_Files/archive/keywords.csv')

<ipython-input-124-72c59eb6ea5a>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_agg = pd.read_csv('/content/gdrive/MyDrive/Bellarmine/CS430_Data_Files/archive/movies_metadata.csv')


Displays first row from each imported dataset

In [ ]:
movie_agg.head(1)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [ ]:
score.head(1)


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529


In [ ]:
cast.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [ ]:
tags.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


# Trimming data

Prints the number of rows for each column in each dataset

In [ ]:
print("movie_agg shape:",movie_agg.shape)
print("movie_agg columns name:", movie_agg.columns)

print("score shape:",score.shape)
print("score columns name:", list(score.columns))

print("cast shape:",cast.shape)
print("columns name:", list(cast.columns))

print("tags shape:",tags.shape)
print("columns name:", list(tags.columns))

movie_agg shape: (45466, 24)
movie_agg columns name: Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
score shape: (26024289, 4)
score columns name: ['userId', 'movieId', 'rating', 'timestamp']
cast shape: (45476, 3)
columns name: ['cast', 'crew', 'id']
tags shape: (46419, 2)
columns name: ['id', 'keywords']


Removes some of the dataframes to save processing time

In [ ]:
movie_agg = pd.read_csv('/content/gdrive/MyDrive/Bellarmine/CS430_Data_Files/archive/movies_metadata.csv') 
movie_agg = movie_agg.iloc[0:10000,:]

<ipython-input-109-f16c7c7e2ea6>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_agg = pd.read_csv('/content/gdrive/MyDrive/Bellarmine/CS430_Data_Files/archive/movies_metadata.csv')


Converts *id* in the listed columns to **int** data type

In [ ]:
tags['id'] = tags['id'].astype('int')
cast['id'] = cast['id'].astype('int')
movie_agg['id'] = movie_agg['id'].astype('int')

Consolidates the **tags** and **cast** dataframes

In [ ]:
movie_agg = movie_agg.merge(cast, on='id')
movie_agg = movie_agg.merge(tags, on='id')
movie_agg.shape

(10048, 27)

Saves **cast**, **crew**, **tags**, and **genres** columns into a new variable called **features**
<br> Changes dataype of **features** to allow for easier parsing

In [ ]:
from ast import literal_eval
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
  movie_agg[feature] = movie_agg[feature].apply(literal_eval)

Grabs the name of the director for each movie from **crew**

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

Returns a list of actors, genres, and keywords

In [ ]:
def get_list(x):
    if isinstance(x, list): 
        names = [i['name'] for i in x] 

        if len(names) > 3:
            names = names[:3]
        return names

    return []

Adds the parsed data back into the dataframe

In [ ]:
movie_agg['director'] = movie_agg['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    movie_agg[feature] = movie_agg[feature].apply(get_list)

movie_agg[['title', 'cast', 'director', 'keywords', 'genres']].head()

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"
3,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devine]",Forest Whitaker,"[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]"
4,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short]",Charles Shyer,"[baby, midlife crisis, confidence]",[Comedy]


# Wordsoup

Converts all words to lowercase and removes all spaces between them

In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x] 
    else:
        
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
          
         

features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
  movie_agg[feature] = movie_agg[feature].apply(clean_data)

Consolidates **features** into a wordsoup by iterating through each row of the dataframe and adding a space inbetween words

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

movie_agg['soup'] = movie_agg.apply(create_soup, axis=1)
movie_agg[['title', 'soup', 'cast', 'director', 'keywords', 'genres']].head()

,title,soup,cast,director,keywords,genres
0,Toy Story,jealousy toy boy tomhanks timallen donrickles ...,"[tomhanks, timallen, donrickles]",johnlasseter,"[jealousy, toy, boy]","[animation, comedy, family]"
1,Jumanji,boardgame disappearance basedonchildren'sbook ...,"[robinwilliams, jonathanhyde, kirstendunst]",joejohnston,"[boardgame, disappearance, basedonchildren'sbook]","[adventure, fantasy, family]"
2,Grumpier Old Men,fishing bestfriend duringcreditsstinger walter...,"[waltermatthau, jacklemmon, ann-margret]",howarddeutch,"[fishing, bestfriend, duringcreditsstinger]","[romance, comedy]"
3,Waiting to Exhale,basedonnovel interracialrelationship singlemot...,"[whitneyhouston, angelabassett, lorettadevine]",forestwhitaker,"[basedonnovel, interracialrelationship, single...","[comedy, drama, romance]"
4,Father of the Bride Part II,baby midlifecrisis confidence stevemartin dian...,"[stevemartin, dianekeaton, martinshort]",charlesshyer,"[baby, midlifecrisis, confidence]",[comedy]


Captures user input and and creates wordsoup for building recommendations

In [ ]:
def get_genres():
  genres = input("What movie genre would you like to watch? (type 'skip' to skip) ")
  return genres

def get_actors():
  actors = input("What actor would you like to see? (type 'skip' to skip) ")
  return actors

def get_directors():
  directors = input("Write the name of a director that you like. (type 'skip' to skip) ")
  return directors

def get_keywords():
  keywords = input("What are some keywords that describe your ideal movie? (e.g. crime, romance, action) (type 'skip' to skip) ")
  keywords = " ".join(["".join(n.split()) for n in keywords.lower().split(',')])
  return keywords

def get_searchTerms():
  searchTerms = [] 
  genres = get_genres()
  if genres != 'skip':
    searchTerms.append(genres)

  actors = get_actors()
  if actors != 'skip':
    searchTerms.append(actors)

  directors = get_directors()
  if directors != 'skip':
    searchTerms.append(directors)

  keywords = get_keywords()
  if keywords != 'skip':
    searchTerms.append(keywords)
  
  return searchTerms

Executes machine learning function and returns recommendations

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def make_recommendation(movie_agg=movie_agg):
  new_row = movie_agg.iloc[-1,:].copy() 
  

  searchTerms = get_searchTerms()  
  new_row.iloc[-1] = " ".join(searchTerms) 
  
  movie_agg = movie_agg.append(new_row)
  
  count = CountVectorizer(stop_words='english')
  count_matrix = count.fit_transform(movie_agg['soup'])

  cosine_sim2 = cosine_similarity(count_matrix, count_matrix) 
  
  sim_scores = list(enumerate(cosine_sim2[-1,:]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  ranked_titles = []
  for i in range(1, 11):
    indx = sim_scores[i][0]
    ranked_titles.append([movie_agg['title'].iloc[indx], movie_agg['imdb_id'].iloc[indx]])
  
  return ranked_titles

make_recommendation()

KeyboardInterrupt: ignored